In [1]:
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers import Dense, Dropout, LSTM, Embedding, Reshape, Input, InputLayer, Concatenate
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model

Using TensorFlow backend.


In [5]:
(lambda i,l: (i,l))(1,2)

(1, 2)

In [12]:
n_samples = 20000
sample_inputs = []
sample_outputs = []
n_tokens = 61

patterns = [
    lambda i,l: list(range(i, i+l)),
    lambda i,l: list(range(i, i-l, -1))
]

for n in range(n_samples):
    sample_in = []
    sample_out = []
    for line_number,length in enumerate([5,7,5]):
        i = np.random.randint(10,50)
        #line = list(range(i, i+3))
        pattern = np.random.randint(len(patterns))
        line = patterns[pattern](i,length)
        '''if line_number == 0:
            sample_in.append([line[0]] + line)
        else:
            sample_in.append([0] + line)'''
        
        sample_in.append([line[0]] + line + [60]*(9-length))
            
        sample_out.append(line + [60]*(10-length))
        
        
        #sample.append(np.arange(i, i+3))
    #samples.append(sample)
    sample_inputs.append(sample_in)
    sample_outputs.append(sample_out)
    
X = np_utils.to_categorical(sample_inputs, num_classes=n_tokens)
Y = np_utils.to_categorical(sample_outputs, num_classes=n_tokens)

In [13]:
sample_inputs

[[[34, 34, 33, 32, 31, 30, 60, 60, 60, 60],
  [47, 47, 46, 45, 44, 43, 42, 41, 60, 60],
  [19, 19, 18, 17, 16, 15, 60, 60, 60, 60]],
 [[49, 49, 50, 51, 52, 53, 60, 60, 60, 60],
  [20, 20, 19, 18, 17, 16, 15, 14, 60, 60],
  [36, 36, 37, 38, 39, 40, 60, 60, 60, 60]],
 [[29, 29, 28, 27, 26, 25, 60, 60, 60, 60],
  [41, 41, 42, 43, 44, 45, 46, 47, 60, 60],
  [40, 40, 39, 38, 37, 36, 60, 60, 60, 60]],
 [[29, 29, 30, 31, 32, 33, 60, 60, 60, 60],
  [32, 32, 31, 30, 29, 28, 27, 26, 60, 60],
  [38, 38, 39, 40, 41, 42, 60, 60, 60, 60]],
 [[27, 27, 28, 29, 30, 31, 60, 60, 60, 60],
  [40, 40, 39, 38, 37, 36, 35, 34, 60, 60],
  [42, 42, 43, 44, 45, 46, 60, 60, 60, 60]],
 [[17, 17, 16, 15, 14, 13, 60, 60, 60, 60],
  [11, 11, 12, 13, 14, 15, 16, 17, 60, 60],
  [47, 47, 46, 45, 44, 43, 60, 60, 60, 60]],
 [[36, 36, 37, 38, 39, 40, 60, 60, 60, 60],
  [28, 28, 29, 30, 31, 32, 33, 34, 60, 60],
  [44, 44, 45, 46, 47, 48, 60, 60, 60, 60]],
 [[32, 32, 33, 34, 35, 36, 60, 60, 60, 60],
  [40, 40, 41, 42, 43, 44

In [14]:
sample_outputs

[[[34, 33, 32, 31, 30, 60, 60, 60, 60, 60],
  [47, 46, 45, 44, 43, 42, 41, 60, 60, 60],
  [19, 18, 17, 16, 15, 60, 60, 60, 60, 60]],
 [[49, 50, 51, 52, 53, 60, 60, 60, 60, 60],
  [20, 19, 18, 17, 16, 15, 14, 60, 60, 60],
  [36, 37, 38, 39, 40, 60, 60, 60, 60, 60]],
 [[29, 28, 27, 26, 25, 60, 60, 60, 60, 60],
  [41, 42, 43, 44, 45, 46, 47, 60, 60, 60],
  [40, 39, 38, 37, 36, 60, 60, 60, 60, 60]],
 [[29, 30, 31, 32, 33, 60, 60, 60, 60, 60],
  [32, 31, 30, 29, 28, 27, 26, 60, 60, 60],
  [38, 39, 40, 41, 42, 60, 60, 60, 60, 60]],
 [[27, 28, 29, 30, 31, 60, 60, 60, 60, 60],
  [40, 39, 38, 37, 36, 35, 34, 60, 60, 60],
  [42, 43, 44, 45, 46, 60, 60, 60, 60, 60]],
 [[17, 16, 15, 14, 13, 60, 60, 60, 60, 60],
  [11, 12, 13, 14, 15, 16, 17, 60, 60, 60],
  [47, 46, 45, 44, 43, 60, 60, 60, 60, 60]],
 [[36, 37, 38, 39, 40, 60, 60, 60, 60, 60],
  [28, 29, 30, 31, 32, 33, 34, 60, 60, 60],
  [44, 45, 46, 47, 48, 60, 60, 60, 60, 60]],
 [[32, 33, 34, 35, 36, 60, 60, 60, 60, 60],
  [40, 41, 42, 43, 44, 45

In [15]:
latent_dim = 128

# Keras Model

inputs = [Input(shape=(None, n_tokens)) for i in range(3)]
lstms = [LSTM(latent_dim, return_state=True, return_sequences=True) for i in range(3)]

lstm_out, lstm_h, lstm_c = [None, None, None], [None, None, None], [None, None, None]
for i in range(3):
    if i > 0:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i], initial_state=[lstm_h[i-1], lstm_c[i-1]])
    else:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i])
        
denses = [Dense(n_tokens, activation='softmax') for i in range(3)]
outputs = [denses[i](lstm_out[i]) for i in range(3)]

model = Model(inputs, outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary(line_length=200)

________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                      Output Shape                                Param #                 Connected to                                                      
input_1 (InputLayer)                                              (None, None, 61)                            0                                                                                         
________________________________________________________________________________________________________________________________________________________________________________________________________
lstm_1 (LSTM)                                                     [(None, None, 128), (None, 128), (None, 128 97280                   input_1[0][0]                                                 

In [16]:
model.fit([X[:,0,:], X[:,1,:], X[:,2,:]], [Y[:,0,:], Y[:,1,:], Y[:,2,:]], batch_size=64, epochs=20)

Epoch 1/20
20000/20000 [==============================] - 9s 453us/step - loss: 5.5482 - dense_1_loss: 1.7625 - dense_2_loss: 2.1922 - dense_3_loss: 1.5936
Epoch 2/20
20000/20000 [==============================] - 8s 395us/step - loss: 2.0223 - dense_1_loss: 0.8121 - dense_2_loss: 0.7105 - dense_3_loss: 0.4997
Epoch 3/20
20000/20000 [==============================] - 8s 382us/step - loss: 0.6703 - dense_1_loss: 0.3713 - dense_2_loss: 0.1768 - dense_3_loss: 0.1222
Epoch 4/20
20000/20000 [==============================] - 8s 375us/step - loss: 0.4036 - dense_1_loss: 0.2138 - dense_2_loss: 0.1023 - dense_3_loss: 0.0875
Epoch 5/20
20000/20000 [==============================] - 7s 370us/step - loss: 0.2940 - dense_1_loss: 0.1255 - dense_2_loss: 0.0893 - dense_3_loss: 0.0791
Epoch 6/20
20000/20000 [==============================] - 7s 370us/step - loss: 0.2525 - dense_1_loss: 0.0899 - dense_2_loss: 0.0859 - dense_3_loss: 0.0767
Epoch 7/20
20000/20000 [==============================] - 8s 376

In [57]:
generator_inputs = [Input(shape=(None, n_tokens), batch_shape=(1,None,n_tokens)) for i in range(3)]
generator_lstms = [LSTM(latent_dim, return_state=True, return_sequences=True, stateful=True) for i in range(3)]
generator_lstm_out, generator_lstm_h, generator_lstm_c = [None, None, None], [None, None, None], [None, None, None]
generator_denses = [Dense(n_tokens, activation='softmax') for i in range(3)]

# Hook up line 0
generator_lstm_out[0], generator_lstm_h[0], generator_lstm_c[0] = generator_lstms[0](
    generator_inputs[0]
)
generator_outputs = [generator_denses[0](generator_lstm_out[0])]
generator_models = [Model(generator_inputs[0], [generator_outputs[0], generator_lstm_h[0], generator_lstm_c[0]])]

# Set weights for line 0
generator_lstms[0].set_weights(lstms[0].get_weights())
generator_denses[0].set_weights(denses[0].get_weights())

generator_models[0].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (1, None, 61)             0         
_________________________________________________________________
lstm_12 (LSTM)               [(1, None, 128), (1, 128) 97280     
_________________________________________________________________
dense_12 (Dense)             (1, None, 61)             7869      
Total params: 105,149
Trainable params: 105,149
Non-trainable params: 0
_________________________________________________________________


In [58]:
generator_inputs_h = [None, Input(shape=(latent_dim,)), Input(shape=(latent_dim,))]
generator_inputs_c = [None, Input(shape=(latent_dim,)), Input(shape=(latent_dim,))]

#generator_inputs.append(Input(shape=(None, n_tokens), batch_shape=(1,None,n_tokens)))
#generator_lstms.append(LSTM(latent_dim, return_state=True, return_sequences=True, stateful=True))


# Hook up lines 1 and 2
for i in [1, 2]:
    generator_lstm_out[i], generator_lstm_h[i], generator_lstm_c[i] = generator_lstms[i](
        generator_inputs[i], initial_state=[generator_inputs_h[i], generator_inputs_c[i]]
    )
    generator_outputs.append(generator_denses[i](generator_lstm_out[i]))
#generator_denses.append(Dense(n_tokens, activation='softmax'))

    generator_models.append(Model(
        [generator_inputs[i], generator_inputs_h[i], generator_inputs_c[i]],
        [generator_outputs[i], generator_lstm_h[i], generator_lstm_c[i]]
    ))

    generator_lstms[i].set_weights(lstms[i].get_weights())
    generator_denses[i].set_weights(denses[i].get_weights())

#generator_model_line1.summary()

In [49]:
#generator_model_line1.reset_states()
#generator_line1_lstm.reset_states()
#generator_lstms[1].reset_states()
line = [23]
#h = np.array([np.zeros(256)])
#c = np.array([np.zeros(256)])

def f():
    while line[-1] != 60:
        yield [np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)), h, c]
        
while line[-1] != 60:
    character, h, c = generator_model_line1.predict_generator(f(), steps=1)
    line.append(np.argmax(character))
    
line

[23, 60]

In [61]:
#LINE 0
generator_models[0].reset_states()
line = [34]

def f():
    #while line[-1] != 60:
    while True:
        yield np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1))
        
#while line[-1] != 60:
while len(line) < 10:
    character, h, c = generator_models[0].predict_generator(f(), steps=1)
    line.append(np.argmax(character))
    
print(line)

# LINE 1
line = [49]

def f():
    while line[-1] != 60:
        yield [np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)), h, c]
        
while line[-1] != 60:
    character, h, c = generator_models[1].predict_generator(f(), steps=1)
    line.append(np.argmax(character))
    
line

[34, 34, 33, 32, 31, 30, 60, 60, 60, 60]


[49, 49, 48, 47, 46, 45, 44, 43, 60]

In [68]:
h,c = None, None
generator_models[0].reset_states()
for i in range(3):
    line = [np.random.randint(10,50)]
    
    def f():
        while True:
            if i > 0:
                yield [np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)), h, c]
            else:
                yield np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1))
        
    while len(line) < 10:
        character, h, c = generator_models[i].predict_generator(f(), steps=1)
        line.append(np.argmax(character))
        
    print(line[1:line.index(60)])

[45, 44, 43, 42, 41]
[22, 23, 24, 25, 26, 27, 28]
[34, 35, 36, 37, 38]


In [33]:
lstms[0].get_weights()

[array([[ 0.0588498 ,  0.07204304, -0.05792206, ..., -0.08177043,
          0.01025008,  0.08220639],
        [ 0.066603  ,  0.0180743 , -0.07614341, ...,  0.08674464,
         -0.09987819, -0.03462665],
        [ 0.00715201,  0.06240957,  0.09057698, ..., -0.07249868,
         -0.02441851,  0.04497065],
        ..., 
        [-0.05160541,  0.04952023, -0.08660023, ..., -0.08238563,
         -0.10212372, -0.04570872],
        [-0.04223401,  0.07268161,  0.0963813 , ..., -0.08047485,
          0.03274512,  0.01094449],
        [ 0.00469505,  0.03440818,  0.10505033, ..., -0.02963178,
          0.04027113, -0.03556556]], dtype=float32),
 array([[ 0.20038167,  0.1037483 , -0.12228944, ...,  0.34388208,
          0.37530544, -0.04092566],
        [ 0.39065197,  0.02621378,  0.12614504, ...,  0.33636111,
         -0.01913496,  0.1178768 ],
        [ 0.34746465,  0.12253693,  0.02178821, ...,  0.46020272,
          0.06336483, -0.06849611],
        ..., 
        [ 0.17965581,  0.33587414, -0

In [34]:
lstms[1].get_weights()

[array([[ 0.02373388,  0.04351699,  0.09366319, ..., -0.07357283,
          0.00148012, -0.07427015],
        [-0.02235256,  0.06999942, -0.05884961, ..., -0.05237408,
         -0.0445431 , -0.06093381],
        [ 0.04001717, -0.05314052, -0.07992874, ...,  0.06541879,
          0.0137012 ,  0.00431497],
        ..., 
        [ 0.01736447, -0.05911503, -0.00703574, ...,  0.00389478,
         -0.04105749,  0.07562602],
        [-0.04369491, -0.02797005, -0.03645102, ..., -0.03725483,
          0.07084001, -0.00418603],
        [-0.00543394, -0.04295987,  0.11130186, ..., -0.25775337,
         -0.07856624,  0.02195806]], dtype=float32),
 array([[ 0.16149174, -0.01383852, -0.04173021, ...,  0.16967967,
         -0.18976605,  0.06395357],
        [-0.04755745,  0.01412346,  0.00423123, ...,  0.18604733,
         -0.05554095,  0.12966727],
        [ 0.18584771, -0.02389204, -0.08125761, ...,  0.18993607,
         -0.00577669,  0.06357786],
        ..., 
        [ 0.06819594,  0.19305532,  0

In [36]:
generator_lstms[1].get_weights()

[array([[ 0.02373388,  0.04351699,  0.09366319, ..., -0.07357283,
          0.00148012, -0.07427015],
        [-0.02235256,  0.06999942, -0.05884961, ..., -0.05237408,
         -0.0445431 , -0.06093381],
        [ 0.04001717, -0.05314052, -0.07992874, ...,  0.06541879,
          0.0137012 ,  0.00431497],
        ..., 
        [ 0.01736447, -0.05911503, -0.00703574, ...,  0.00389478,
         -0.04105749,  0.07562602],
        [-0.04369491, -0.02797005, -0.03645102, ..., -0.03725483,
          0.07084001, -0.00418603],
        [-0.00543394, -0.04295987,  0.11130186, ..., -0.25775337,
         -0.07856624,  0.02195806]], dtype=float32),
 array([[ 0.16149174, -0.01383852, -0.04173021, ...,  0.16967967,
         -0.18976605,  0.06395357],
        [-0.04755745,  0.01412346,  0.00423123, ...,  0.18604733,
         -0.05554095,  0.12966727],
        [ 0.18584771, -0.02389204, -0.08125761, ...,  0.18993607,
         -0.00577669,  0.06357786],
        ..., 
        [ 0.06819594,  0.19305532,  0